In [1]:
import numpy as np
from keras import utils, callbacks
from keras.models import Sequential
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split, StratifiedKFold
#from keras.preprocessing.image import load_img, img_to_array
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_directory = "../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train"
#test_directory = "../input/asl-alphabet/asl_alphabet_test/asl_alphabet_test"

In [3]:
classes = 29
batch = 256
epochs = 50
lnr = 0.0001

In [4]:
gen = ImageDataGenerator(rescale=1./255, validation_split=0.1) 
train = gen.flow_from_directory("../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train", 
                                                            target_size=(64, 64), subset="training")
val = gen.flow_from_directory("../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train", 
                                                            target_size=(64, 64), subset="validation")

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [5]:
adam = Adam(learning_rate=lnr)
model = Sequential()
model.add(VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(29, activation='softmax'))

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'] )

58892288/58889256 [==============================] - 1s 0us/step


In [6]:
earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", 
                                        patience=5, restore_best_weights = True)

history = model.fit(train, validation_data = val, epochs=epochs, shuffle=True, verbose=1, 
                    callbacks = [earlystopping])

Epoch 1/50
2447/2447 [==============================] - 483s 196ms/step - loss: 0.4633 - accuracy: 0.8639 - val_loss: 0.1054 - val_accuracy: 0.9672
Epoch 2/50
2447/2447 [==============================] - 144s 59ms/step - loss: 0.0213 - accuracy: 0.9942 - val_loss: 0.0943 - val_accuracy: 0.9767
Epoch 3/50
2447/2447 [==============================] - 146s 60ms/step - loss: 0.0102 - accuracy: 0.9975 - val_loss: 0.0406 - val_accuracy: 0.9892
Epoch 4/50
2447/2447 [==============================] - 148s 61ms/step - loss: 0.0209 - accuracy: 0.9954 - val_loss: 0.1278 - val_accuracy: 0.9679
Epoch 5/50
2447/2447 [==============================] - 147s 60ms/step - loss: 0.0103 - accuracy: 0.9976 - val_loss: 0.0596 - val_accuracy: 0.9837
Epoch 6/50
2447/2447 [==============================] - 148s 61ms/step - loss: 0.0068 - accuracy: 0.9986 - val_loss: 0.1134 - val_accuracy: 0.9761
Epoch 7/50
2447/2447 [==============================] - 148s 61ms/step - loss: 0.0070 - accuracy: 0.9987 - val_loss: 

In [7]:
test = ImageDataGenerator(rescale=1./255).flow_from_directory("../input/asl-alphabet/asl_alphabet_test", 
                                                            target_size=(64, 64), class_mode=None)
pred = model.predict(test)
pred = np.argmax(pred, axis=1)
pred

Found 28 images belonging to 1 classes.


array([ 1,  4,  0, 11,  6, 21,  3, 20, 23,  8, 24,  2, 19, 18, 27,  7, 28,
       15, 10, 16,  9, 12, 13,  5, 17, 25, 22, 14])

In [8]:
model.save('final_model.h5')